#### Modules

In [1]:
import requests
import json, os, pprint
import getpass
import base64
import urllib.parse

#import utils.Viya_REST_API

requests.packages.urllib3.disable_warnings()

#### Parameters

In [3]:
hostport="https://sasserver.demo.sas.com:443"
path = "/data/sascode/viya_dataOps"
target_root = '/public/VSCODE'
target_folder = 'viya_DataOps'


#### Functions

In [4]:
#### Get ID of folder
def get_folder_id (hostport, authToken, folder_path):
    url = hostport + '/folders/folders/@item?path=' + folder_path

    headers = {
      'Accept': 'application/vnd.sas.content.folder+json',  
      'Authorization': 'Bearer ' + authToken
    }

    response = requests.get( url, headers = headers, verify=False)
    # target_id = response.json().get("id")
    # print(target_id)
    return response

#### Delete folder in recursively, both folder and files
def delete_folder_recursively(hostport, authToken, target_id):

    if target_id is None:
      print("Target folder does not seem to exist, nothing deleted")
      return
    # Get the members of the target folder
    url = hostport + '/folders/folders/' + target_id + '/members?recursive=true'
    headers = {
      'Authorization': 'Bearer ' + authToken
    }
    
    response = requests.get( url, headers = headers, verify=False)
    if response.reason == 'OK':
        # Delete all the files in the target folder 
        item_list = []
        items = response.json().get("items")
        item_list = [x["uri"] for x in items if x['contentType'] == 'file']
        #print(item_list)
        for i in item_list:
            url = hostport + i
            response = requests.delete( url, headers = headers, verify=False)
            # pprint.pprint(result.json())

        # Delete the target foldwer recursively
        url = hostport + '/folders/folders/' + target_id + '?recursive=true';
        response = requests.delete( url, headers = headers, verify=False)
        #pprint.pprint(result.json())
    else:
        print("Target folder does not seem to exist, nothing deleted")
    
    return response

#### Create folder
def create_folder (hostport, authToken, parent_uri, current_dir):
    url = hostport + '/folders/folders'
    
    headers = {
      'Content-Type': 'application/vnd.sas.content.folder+json',
      'Accept': 'application/vnd.sas.content.folder+json',
      'Authorization': 'Bearer ' + authToken
    }
    
    body = {
       'parentFolderUri': '/folders/folders/' + parent_uri,
       'name': current_dir
    }
    
    response = requests.post( url, 
                              data = json.dumps(body), 
                              headers = headers,
                              verify = False)
    return response

#### Upload file
def upload_file (hostport, authToken, infile, name):
    url = hostport + '/files/files'

    headers = {
      'Content-Type': 'multipart/form-data',
      'Accept': 'application/vnd.sas.file+json',
      'Content-Disposition': 'attachment; file=' + name + '; filename=' + name,
      'Content-Type': 'text/plain',
      'Content-Transfer-Encoding': 'binary',
      'Authorization': 'Bearer ' + authToken  
    }

    with open(infile, 'r') as file:
        body = file.read().rstrip()
    
    response = requests.post(url, 
                      headers = headers,
                      data = body,
                      verify=False)
    # id_file = r.json().get('id')
    # print(id_file)   
    return response

#### Associate file with folder
def associate_file_with_folder (hostport, authToken, folder_id, file_id, name):

    url = hostport + '/folders/folders/' + folder_id + '/members'

    headers = {
      'Content-Type': 'application/vnd.sas.content.folder.member+json',  
      'Authorization': 'Bearer ' + authToken
    }

    body = {
        "name": name,
        "uri": "/files/files/" + file_id,
        "type": "reference",
        "contentType": "file"
    }

    response = requests.post( url, 
                      headers = headers,
                      data = json.dumps(body),
                      verify=False)
    return response



#### Get token

In [5]:
#Get Authorization Token
authUri="/SASLogon/oauth/token"

headersAuth={
    "accept":"application/json",
    "content-type":"application/x-www-form-urlencoded",
    "Authorization":"Basic c2FzLmVjOg=="
}
authToken=""
user=""
password=""
notAuthed=True

while notAuthed :
    user = input("Enter user ID: ")
    password =  urllib.parse.quote(getpass.getpass('Enter password for user %s:' % user))
    authBody='grant_type=password&username=' + user + '&password=' + password
    authReturn = requests.post(hostport+authUri, data=authBody, headers=headersAuth, verify=False)
    if authReturn.status_code == requests.codes.ok :
        authToken = authReturn.json()['access_token']
        notAuthed = False
    else :
        print("Please enter a valid user ID and password.")
    
password = ""

#### Get ID of root and target folder (if exist)

In [8]:
folder_path = target_root + '/' + target_folder
response = get_folder_id (hostport, authToken, folder_path)
target_id = response.json().get("id")
print(target_id)

folder_path = target_root
response = get_folder_id (hostport, authToken, folder_path)
root_id = response.json().get("id")
print(root_id)

44953de2-92f2-459f-9cc7-ca26932b332d
4d0c376b-5fe0-4124-80ac-93abf867fb08


#### Delete target folder recursively

In [9]:
delete_folder_recursively(hostport, authToken, target_id)


<Response [204]>

#### Create folders and files

In [10]:
# Add public_id to dictionary
list = {'root': root_id}

fname = []
# Loop through directories
for root,d_names,f_names in os.walk(path):
    # Root path is replaced with target folder
    root_stripped = root.replace(path, target_folder)
    if root_stripped.find('.') != -1: 
        continue
    print(root_stripped)
    root_parts = root_stripped.split('/')
    number_parts = len(root_parts)
    current_dir = root_parts[-1]
    if number_parts == 1:
        parent_dir = 'root'
    elif number_parts > 1:
        parent_dir = root_parts[-2]

    # Lookup parent_id
    parent_uri = list[parent_dir]
    
    # Create subfolder
    response = create_folder (hostport, authToken, parent_uri, current_dir)
    folder_id = response.json().get("id")
    
    # Add current_dir to dictionary
    list[current_dir] = response.json().get("id")
    
    for one_file in f_names:
        infile = root + '/' + one_file
        response = upload_file (hostport, authToken, infile, one_file)
        file_id = response.json().get("id")
        associate_file_with_folder (hostport, authToken, folder_id, file_id, one_file)


viya_DataOps
viya_DataOps/utils
viya_DataOps/programs
viya_DataOps/macros
viya_DataOps/jenkins
